# Intalling & Importing Libraries

In [8]:
!pip install fuzzywuzzy
import pandas as pd
import numpy as np
import nltk
from nltk.corpus import stopwords
nltk.download('wordnet')
nltk.download('punkt')
stop_words = stopwords.words('english')
from nltk import word_tokenize
from nltk.stem import WordNetLemmatizer
from nltk.corpus import stopwords
import re
import gensim
from tqdm import tqdm
from scipy.stats import skew, kurtosis
from scipy.spatial.distance import cosine, cityblock, jaccard, canberra, euclidean, minkowski, braycurtis
from google.colab import drive
drive.mount('/content/drive')

[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\ankit\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\ankit\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!


# Function For Importing Dataset

In [0]:
def importDataset():
    data = pd.read_csv('/content/drive/My Drive/majorProject/quora_duplicate_questions.tsv', sep='\t')
    data = data.drop(['id', 'qid1', 'qid2'], axis=1)
    return data

# WMD

In [0]:
def wmd(s1, s2):
    
    lemma  = WordNetLemmatizer()
    s1 = str(s1).lower().split()
    s2 = str(s2).lower().split()
    s1 = [lemma.lemmatize(w) for w in s1 if w not in stopwords.words('english')]
    s2 = [lemma.lemmatize(w) for w in s2 if w not in stopwords.words('english')]
    return model.wmdistance(s1, s2)

# Sent2Vec

In [0]:
def sent2vec(s):
    words = str(s).lower()
    words = word_tokenize(words)
    words = [w for w in words if not w in stopwords.words('english')]
    words = [w for w in words if w.isalpha()]
    M = []
    for w in words:
        try:
            M.append(model[w])
        except:
            continue
    M = np.array(M)
    v = M.sum(axis=0)
    return v / np.sqrt((v ** 2).sum())

# Load Dataset

In [13]:
dataset = importDataset()
dataset.dropna(axis=0, how='any', inplace=True)
dataset.head(5)

,question1,question2,is_duplicate
0,What is the step by step guide to invest in sh...,What is the step by step guide to invest in sh...,0
1,What is the story of Kohinoor (Koh-i-Noor) Dia...,What would happen if the Indian government sto...,0
2,How can I increase the speed of my internet co...,How can Internet speed be increased by hacking...,0
3,Why am I mentally very lonely? How can I solve...,Find the remainder when [math]23^{24}[/math] i...,0
4,"Which one dissolve in water quikly sugar, salt...",Which fish would survive in salt water?,0


# Loading GoogleNewsVector

In [0]:
model = gensim.models.KeyedVectors.load_word2vec_format('/content/drive/My Drive/majorProject/GoogleNews-vectors-negative300.bin.gz', binary=True)

# Calculating Normalized_WMD

In [0]:
model.init_sims(replace=True)
dataset['norm_wmd'] = dataset.apply(lambda x: wmd(x['question1'], x['question2']), axis=1)

In [0]:
dataset.to_csv('FEATURE_II.csv', index=False)

In [0]:
question1_vectors = np.zeros((dataset.shape[0], 300))
question2_vectors  = np.zeros((dataset.shape[0], 300))
error_count = 0

# Calculating Question Vectors

In [31]:
for i, q in tqdm(enumerate(dataset.question1.values)):
    question1_vectors[i, :] = sent2vec(q)

249it [00:00, 477.70it/s]/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:14: RuntimeWarning: invalid value encountered in double_scalars
  
404287it [11:23, 591.18it/s]


In [32]:
for i, q in tqdm(enumerate(dataset.question2.values)):
    question2_vectors[i, :] = sent2vec(q)

203it [00:00, 486.98it/s]/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:14: RuntimeWarning: invalid value encountered in double_scalars
  
404287it [11:35, 580.98it/s]


In [34]:
dataset['cosine_distance'] = [cosine(x, y) for (x, y) in zip(np.nan_to_num(question1_vectors),
                                                          np.nan_to_num(question2_vectors))]

dataset['cityblock_distance'] = [cityblock(x, y) for (x, y) in zip(np.nan_to_num(question1_vectors),
                                                          np.nan_to_num(question2_vectors))]

dataset['jaccard_distance'] = [jaccard(x, y) for (x, y) in zip(np.nan_to_num(question1_vectors),
                                                          np.nan_to_num(question2_vectors))]

dataset['canberra_distance'] = [canberra(x, y) for (x, y) in zip(np.nan_to_num(question1_vectors),
                                                          np.nan_to_num(question2_vectors))]

dataset['euclidean_distance'] = [euclidean(x, y) for (x, y) in zip(np.nan_to_num(question1_vectors),
                                                          np.nan_to_num(question2_vectors))]

dataset['minkowski_distance'] = [minkowski(x, y, 3) for (x, y) in zip(np.nan_to_num(question1_vectors),
                                                          np.nan_to_num(question2_vectors))]

dataset['braycurtis_distance'] = [braycurtis(x, y) for (x, y) in zip(np.nan_to_num(question1_vectors),
                                                          np.nan_to_num(question2_vectors))]

dataset['skew_q1vec'] = [skew(x) for x in np.nan_to_num(question1_vectors)]
dataset['skew_q2vec'] = [skew(x) for x in np.nan_to_num(question2_vectors)]
dataset['kur_q1vec'] = [kurtosis(x) for x in np.nan_to_num(question1_vectors)]
dataset['kur_q2vec'] = [kurtosis(x) for x in np.nan_to_num(question2_vectors)]

/usr/local/lib/python3.6/dist-packages/scipy/spatial/distance.py:702: RuntimeWarning: invalid value encountered in double_scalars
  dist = 1.0 - uv / np.sqrt(uu * vv)
/usr/local/lib/python3.6/dist-packages/scipy/spatial/distance.py:1160: RuntimeWarning: invalid value encountered in double_scalars
  return l1_diff.sum() / l1_sum.sum()


# Saving Feature

In [ ]:
dataset.to_csv('features_I.csv', index=False)